# Data Preparation and Setup

The cells below prepare the TAZ and Network Dataset datasets used for all ATO calculations. 

Before you get started:

1. Download [Wasatch Front TAZs in File Geodatabase format from WFRC](https://data.wfrc.org/datasets/wfrc::access-to-opportunities-work-related-taz-based/about). Extract to `taz.gdb` and place in the `shp` folder
4. Update the files and fields in the cell below

In [1]:
# source files and fields - update this before running!
source_taz = r'shp\taz.gdb\ATO'
hh_source_field = 'HH_19' # field containing TAZ household count
job_source_field = 'JOB_19' # field containing TAZ job count

In [2]:
import sys
import os
import arcpy
import shutil

import pandas as pd
from arcgis.features import SpatialDataFrame

src = os.path.join(os.path.abspath("."), 'src')
if src not in sys.path:
    sys.path.append(src)
    
from ato_tools import ato

# Set the XYResolution environment to a linear unit
arcpy.env.XYResolution = "0.01 Meters"
arcpy.env.XYTolerance = "0.1 Meters"

# UTM 12N
crs = 'PROJCS["NAD_1983_UTM_Zone_12N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-111.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'
base_path = os.path.abspath(".")

base_gdb = os.path.join(base_path, "baseline.gdb")



## TAZ Setup

Configure the TAZ table and centroid locations.

In [24]:
# Copy TAZ Polygons to Baseline - keeping only CO_TAZID Field
arcpy.conversion.FeatureClassToFeatureClass(
    source_taz, 
    base_gdb, 
    "taz", 
    '', 
    r'CO_TAZID "CO_TAZID" true true false 4 Long 0 0,First,#,shp\taz.gdb\ATO,CO_TAZID,-1,-1', 
    ''
)

<Result 'C:\\wfrc\\ato\\baseline.gdb\\taz'>

In [25]:
#if not "square_meters_taz" in [field.name for field in arcpy.ListFields("TAZ")]:
arcpy.management.CalculateGeometryAttributes(
    r"baseline.gdb\taz", "square_meters_taz AREA", '', "SQUARE_METERS"
)

<Result 'baseline.gdb\\taz'>

In [26]:
# copy table of CO_TAZID, HH, and JOB to baseline gdb
taz_table = pd.DataFrame.spatial.from_featureclass(source_taz)
taz_table['HH'] = taz_table[hh_source_field]
taz_table['JOB'] = taz_table[job_source_field]
taz_table.drop(columns=taz_table.columns.difference(['CO_TAZID', 'HH', 'JOB']), inplace=True)
taz_table.spatial.to_table(os.path.join(base_gdb, "taz_table"))

'C:\\wfrc\\ato\\baseline.gdb\\taz_table'

In [27]:
# calculate TAZ centroids
# (the arcpy "Feature to Point" tool makes this easier but requires an advanced license)
    
arcpy.management.CalculateGeometryAttributes(
    os.path.join(base_gdb, "taz"), 
    "x CENTROID_X;y CENTROID_Y", 
    '', '', crs, "SAME_AS_INPUT"
)

arcpy.management.XYTableToPoint(
    os.path.join(base_gdb, "taz"), 
    os.path.join(base_gdb, "taz_centroids"), 
    'x', 'y', None, crs
)

arcpy.management.MakeFeatureLayer(
    os.path.join(base_gdb, "taz_centroids"), 
    "taz_centroids"
)

arcpy.management.DeleteField(os.path.join(base_gdb, "taz"), ['x','y'])

<Result 'C:\\wfrc\\ato\\baseline.gdb\\taz'>

Note, snapping TAZ centroids to the network can introduce some unintended variation in ATO between TAZs if the centroid snaps to a location only accessible via a circuitous route, to a roadway outside of the TAZ boundaries, etc. Also note, centroids are snapped to auto network. Additional snapping happens within the network solver configuration for transit and bicycle routes. None of this should materially affect the estimated change in ATO for a given project.

In [ ]:
# snap taz centroids to network
# reqires ArcGIS Standard or Advanced
# this step can be skipped but is recommended
arcpy.management.MakeFeatureLayer(
    os.path.join(base_gdb, r"NetworkDataset\BikePedAuto"), 
    "bpa_snap"
)
arcpy.management.SelectLayerByAttribute(
      "bpa_snap", "NEW_SELECTION", "AutoNetwork = 'Y' And VERT_LEVEL = '0' And CartoCode NOT IN ('1 Interstates')"
)
arcpy.edit.Snap("taz_centroids", "bpa_snap End '10000 Feet'")

In [28]:
# Finally, create a testing layer with 25 randomly selected centroids
import random
arcpy.management.SelectLayerByAttribute('taz_centroids', "CLEAR_SELECTION")

feature_count = int(arcpy.management.GetCount('taz_centroids').getOutput(0))
rnd_set = set([]) 
while len(rnd_set) < 25: 
    rnd_set.add(random.randint(0, feature_count-1))
where = 'OBJECTID in ({0})'.format(','.join(map(str,rnd_set)))
arcpy.management.SelectLayerByAttribute('taz_centroids', "NEW_SELECTION", where)

arcpy.conversion.FeatureClassToFeatureClass(
    'taz_centroids', 
    base_gdb, 
    "taz_centroids_sample"
)

arcpy.management.SelectLayerByAttribute('taz_centroids', "CLEAR_SELECTION")

id,value
0,a Layer object
1,-1


## Baseline Scoring

Calculate ATO for our baseline modal networks.

Finally, we create a copy of the baseline network dataset to use as a template for scenario networks.

Note, typical solve times:

* Driving: 20 minutes
* Transit: 4 - 5 minutes
* Cycling: 4 - 5 minutes

If solve times deviate signficantly, consider rebuilding the network dataset and re-solving. (This is related to the Network Analyst attribute mismatch issue.)

In [34]:
for mode in ['Driving', 'Transit', 'Cycling']:
    ato.skim(
        nd = os.path.join(base_gdb, r'NetworkDataset\NetworkDataset_ND'),
        mode = mode,
        centroids = os.path.join(base_gdb, 'taz_centroids'),
        out_table = os.path.join(base_gdb, 'skim_' + mode.lower())
    )

Solving skim using Transit network for C:\wfrc\ato\baseline.gdb\NetworkDataset\NetworkDataset_ND .
Skim matrix written to skim_transit
Skim Matrix Solve Time (mins): 4.18
Solving skim using Cycling network for C:\wfrc\ato\baseline.gdb\NetworkDataset\NetworkDataset_ND .
Skim matrix written to skim_cycling
Skim Matrix Solve Time (mins): 4.34


In [6]:
for mode in ['driving', 'transit', 'cycling']:
    ato.score(
        skim_matrix = os.path.join(base_gdb, 'skim_' + mode),
        taz_table = os.path.join(base_gdb, 'taz_table'),
        out_table = os.path.join(base_gdb, 'ato_' + mode),
        job_per_hh = 1.808755
    )

Scores written to C:\wfrc\ato\baseline.gdb\ato_driving
Network ATO: 612920610.0
Scores written to C:\wfrc\ato\baseline.gdb\ato_transit
Network ATO: 45745429.0
Scores written to C:\wfrc\ato\baseline.gdb\ato_cycling
Network ATO: 90178234.0


In [7]:
# Create "template" to use for mods

if os.path.isdir(r"scenario\scenario_template.gdb"):
    shutil.rmtree(r"scenario\scenario_template.gdb")

arcpy.management.CreateFileGDB("scenario", "scenario_template")

# Copy our baseline network dataset to our dataset for modification
arcpy.management.Copy(
    r"baseline.gdb\NetworkDataset", 
    r"scenario\scenario_template.gdb\NetworkDataset"
)

arcpy.management.Copy(
    r"baseline.gdb\taz_table", 
    r"scenario\scenario_template.gdb\taz_table"
)

# delete existing network
arcpy.management.Delete(r"scenario\scenario_template.gdb\NetworkDataset\NetworkDataset_ND")

# if open in ArcGIS Pro, remove all layers
try:
    aprx = arcpy.mp.ArcGISProject("CURRENT")
    mp = aprx.listMaps("Map")[0]
    for rmlyr in mp.listLayers():    
        if rmlyr.name not in ['World Topographic Map', 'World Hillshade']:        
            mp.removeLayer(rmlyr)
except OSError:
    pass

<Result 'true'>